# Feature Engineering

The next step is to create features from the raw text so we can train the machine learning models. The steps followed are:

1. **Text Cleaning and Preparation**: cleaning of special characters, downcasing, punctuation signs. possessive pronouns and stop words removal and lemmatization. 
2. **Label coding**: creation of a dictionary to map each category to a code.
3. **Train-test split**: to test the models on unseen data.
4. **Text representation**: use of TF-IDF scores to represent text.

In [1]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

First of all we'll load the dataset:

In [2]:
path_df = "Cars_dataset.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

In [3]:
df.head()

,Manufacturer,Permit Number,DATE,VIN NUMBER,VEHICLE IS CAPABLE OF OPERATING WITHOUT A DRIVER\n(Yes or No),DRIVER PRESENT\n(Yes or No),"DISENGAGEMENT INITIATED BY\n(AV System, Test Driver, Remote Operator, or Passenger)","DISENGAGEMENT\nLOCATION\n(Interstate, Freeway, Highway, Rural Road, Street, or Parking Facility)",DESCRIPTION OF FACTS CAUSING DISENGAGEMENT,Unnamed: 9,Unnamed: 10,id,desc_length
0,Ambarella Corp.,AVT053,3/14/2018,3LN6L5MU7HR609845,No,Yes,test driver,street,Unexpected result from the path planner in the...,NaN,NaN,1,71
1,Ambarella Corp.,AVT053,3/14/2018,3LN6L5MU7HR609845,No,Yes,test driver,street,Unexpected result from the radar based percept...,NaN,NaN,1,81
2,Ambarella Corp.,AVT053,3/14/2018,3LN6L5MU7HR609845,No,Yes,test driver,street,Unexpected result from the path planner in the...,NaN,NaN,1,71
3,Ambarella Corp.,AVT053,3/14/2018,3LN6L5MU7HR609845,No,Yes,test driver,street,Unexpected result from the GPS system in the g...,NaN,NaN,1,69
4,Ambarella Corp.,AVT053,3/15/2018,3LN6L5MU7HR609845,No,Yes,test driver,street,Unexpected result from the GPS system in the g...,NaN,NaN,1,69


And visualize one sample news content:

In [4]:
df.loc[1]['DESCRIPTION OF FACTS CAUSING DISENGAGEMENT']

'Unexpected result from the radar based perception in the given traffic conditions'

## 1. Text cleaning and preparation

### 1.1. Special character cleaning

We can see the following special characters:

* ``\r``
* ``\n``
* ``\`` before possessive pronouns (`government's = government\'s`)
* ``\`` before possessive pronouns 2 (`Yukos'` = `Yukos\'`)
* ``"`` when quoting text

In [5]:
# \r and \n
df['Content_Parsed_1'] = df['DESCRIPTION OF FACTS CAUSING DISENGAGEMENT'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

Regarding 3rd and 4th bullet, although it seems there is a special character, it won't affect us since it is not a *real* character:

In [6]:
text = "Mr Greenspan\'s"
text

"Mr Greenspan's"

In [7]:
# " when quoting text
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')

### 1.2. Upcase/downcase

We'll downcase the texts because we want, for example, `Football` and `football` to be the same word.

In [8]:
# Lowercasing the text
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

### 1.3. Punctuation signs

Punctuation signs won't have any predicting power, so we'll just get rid of them.

In [9]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

<ipython-input-9-cdceb98d92a9>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')


By doing this we are messing up with some numbers, but it's no problem since we aren't expecting any predicting power from them.

### 1.4. Possessive pronouns

We'll also remove possessive pronoun terminations:

In [10]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")

### 1.5. Stemming and Lemmatization

Since stemming can produce output words that don't exist, we'll only use a lemmatization process at this moment. Lemmatization takes into consideration the morphological analysis of the words and returns words that do exist, so it will be more useful for us.

In [11]:
# Downloading punkt and wordnet from NLTK
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pragnareddychalla/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


------------------------------------------------------------


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pragnareddychalla/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [12]:
# Saving the lemmatizer into an object
wordnet_lemmatizer = WordNetLemmatizer()

In order to lemmatize, we have to iterate through every word:

In [13]:
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [14]:
df['Content_Parsed_5'] = lemmatized_text_list

Although lemmatization doesn't work perfectly in all cases (as can be seen in the example below), it can be useful.

### 1.6. Stop words

In [15]:
# Downloading the stop words list
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pragnareddychalla/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))

In [17]:
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

To remove the stop words, we'll handle a regular expression only detecting whole words, as seen in the following example:

In [18]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

We can now loop through all the stop words:

In [19]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

<ipython-input-19-3b7196a1b53b>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')


We have some dobule/triple spaces between words because of the replacements. However, it's not a problem because we'll tokenize by the spaces later.

As an example, we'll show an original news article and its modifications throughout the process:

In [21]:
df.loc[5]['DESCRIPTION OF FACTS CAUSING DISENGAGEMENT']

'Unexpected result from the radar based perception in the given traffic conditions'

1. Special character cleaning

In [22]:
df.loc[5]['Content_Parsed_1']

'Unexpected result from the radar based perception in the given traffic conditions'

2. Upcase/downcase

In [23]:
df.loc[5]['Content_Parsed_2']

'unexpected result from the radar based perception in the given traffic conditions'

3. Punctuation signs

In [24]:
df.loc[5]['Content_Parsed_3']

'unexpected result from the radar based perception in the given traffic conditions'

4. Possessive pronouns

In [25]:
df.loc[5]['Content_Parsed_4']

'unexpected result from the radar based perception in the given traffic conditions'

5. Stemming and Lemmatization

In [26]:
df.loc[5]['Content_Parsed_5']

'unexpected result from the radar base perception in the give traffic condition'

6. Stop words

In [27]:
df.loc[5]['Content_Parsed_6']

'unexpected result   radar base perception   give traffic condition'

Finally, we can delete the intermediate columns:

In [28]:
df.head(1)

,Manufacturer,Permit Number,DATE,VIN NUMBER,VEHICLE IS CAPABLE OF OPERATING WITHOUT A DRIVER\n(Yes or No),DRIVER PRESENT\n(Yes or No),"DISENGAGEMENT INITIATED BY\n(AV System, Test Driver, Remote Operator, or Passenger)","DISENGAGEMENT\nLOCATION\n(Interstate, Freeway, Highway, Rural Road, Street, or Parking Facility)",DESCRIPTION OF FACTS CAUSING DISENGAGEMENT,Unnamed: 9,Unnamed: 10,id,desc_length,Content_Parsed_1,Content_Parsed_2,Content_Parsed_3,Content_Parsed_4,Content_Parsed_5,Content_Parsed_6
0,Ambarella Corp.,AVT053,3/14/2018,3LN6L5MU7HR609845,No,Yes,test driver,street,Unexpected result from the path planner in the...,NaN,NaN,1,71,Unexpected result from the path planner in the...,unexpected result from the path planner in the...,unexpected result from the path planner in the...,unexpected result from the path planner in the...,unexpected result from the path planner in the...,unexpected result path planner give traffi...


In [30]:
df.columns

Index(['Manufacturer', 'Permit Number', 'DATE', 'VIN NUMBER',
       'VEHICLE IS CAPABLE OF OPERATING WITHOUT A DRIVER\n(Yes or No)',
       'DRIVER PRESENT\n(Yes or No)',
       'DISENGAGEMENT INITIATED BY\n(AV System, Test Driver, Remote Operator, or Passenger)',
       'DISENGAGEMENT\nLOCATION\n(Interstate, Freeway, Highway, Rural Road, Street, or Parking Facility)',
       'DESCRIPTION OF FACTS CAUSING DISENGAGEMENT', 'Unnamed: 9',
       'Unnamed: 10', 'id', 'desc_length', 'Content_Parsed_1',
       'Content_Parsed_2', 'Content_Parsed_3', 'Content_Parsed_4',
       'Content_Parsed_5', 'Content_Parsed_6'],
      dtype='object')

In [32]:
list_columns = ["Manufacturer", "Permit Number",'DESCRIPTION OF FACTS CAUSING DISENGAGEMENT', "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})

In [33]:
df.head()

,Manufacturer,Permit Number,DESCRIPTION OF FACTS CAUSING DISENGAGEMENT,Content_Parsed
0,Ambarella Corp.,AVT053,Unexpected result from the path planner in the...,unexpected result path planner give traffi...
1,Ambarella Corp.,AVT053,Unexpected result from the radar based percept...,unexpected result radar base perception gi...
2,Ambarella Corp.,AVT053,Unexpected result from the path planner in the...,unexpected result path planner give traffi...
3,Ambarella Corp.,AVT053,Unexpected result from the GPS system in the g...,unexpected result gps system give traffic ...
4,Ambarella Corp.,AVT053,Unexpected result from the GPS system in the g...,unexpected result gps system give traffic ...


**IMPORTANT:**

We need to remember that our model will gather the latest news articles from different newspapers every time we want. For that reason, we not only need to take into account the peculiarities of the training set articles, but also possible ones that are present in the gathered news articles.

For this reason, possible peculiarities have been studied in the *05. News Scraping* folder.

## 2. Label coding

In [34]:
df['Manufacturer'].unique()

array(['Ambarella Corp.', 'Apex.Ai, Inc.', 'Box Bot Inc.',
       'DiDi Research America, LLC', 'Gatik AI Inc.', 'Intel Corporation',
       'RIDECELL INC', 'ThorDrive, Inc.'], dtype=object)

We'll create a dictionary with the label codification:

In [35]:
category_codes={'Ambarella Corp.':0, 'Apex.Ai, Inc.':1, 'Box Bot Inc.':2,
       'DiDi Research America, LLC':3, 'Gatik AI Inc.':4, 'Intel Corporation':5,
       'RIDECELL INC':6, 'ThorDrive, Inc.':7}

In [36]:
# Category mapping
df['Category_Code'] = df['Manufacturer']
df = df.replace({'Category_Code':category_codes})

In [37]:
df.head()

,Manufacturer,Permit Number,DESCRIPTION OF FACTS CAUSING DISENGAGEMENT,Content_Parsed,Category_Code
0,Ambarella Corp.,AVT053,Unexpected result from the path planner in the...,unexpected result path planner give traffi...,0
1,Ambarella Corp.,AVT053,Unexpected result from the radar based percept...,unexpected result radar base perception gi...,0
2,Ambarella Corp.,AVT053,Unexpected result from the path planner in the...,unexpected result path planner give traffi...,0
3,Ambarella Corp.,AVT053,Unexpected result from the GPS system in the g...,unexpected result gps system give traffic ...,0
4,Ambarella Corp.,AVT053,Unexpected result from the GPS system in the g...,unexpected result gps system give traffic ...,0


## 3. Train - test split

We'll set apart a test set to prove the quality of our models. We'll do Cross Validation in the train set in order to tune the hyperparameters and then test performance on the unseen data of the test set.

In [38]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

Since we don't have much observations (only 2.225), we'll choose a test set size of 15% of the full dataset.

## 4. Text representation

We have various options:

* Count Vectors as features
* TF-IDF Vectors as features
* Word Embeddings as features
* Text / NLP based features
* Topic Models as features

We'll use **TF-IDF Vectors** as features.

We have to define the different parameters:

* `ngram_range`: We want to consider both unigrams and bigrams.
* `max_df`: When building the vocabulary ignore terms that have a document
    frequency strictly higher than the given threshold
* `min_df`: When building the vocabulary ignore terms that have a document
    frequency strictly lower than the given threshold.
* `max_features`: If not None, build a vocabulary that only consider the top
    max_features ordered by term frequency across the corpus.

See `TfidfVectorizer?` for further detail.

It needs to be mentioned that we are implicitly scaling our data when representing it as TF-IDF features with the argument `norm`.

In [39]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

We have chosen these values as a first approximation. Since the models that we develop later have a very good predictive power, we'll stick to these values. But it has to be mentioned that different combinations could be tried in order to improve even more the accuracy of the models.

In [40]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(385, 78)
(69, 78)


Please note that we have fitted and then transformed the training set, but we have **only transformed** the **test set**.

We can use the Chi squared test in order to see what unigrams and bigrams are most correlated with each category:

In [41]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")


# 'Ambarella Corp.' category:
  . Most correlated unigrams:
. system
. give
. result
. unexpected
. condition
  . Most correlated bigrams:
. traffic condition
. unexpected result

# 'Apex.Ai, Inc.' category:
  . Most correlated unigrams:
. reckless
. set
. participant
. deficiency
. sensor
  . Most correlated bigrams:
. drive traffic
. traffic participant

# 'Box Bot Inc.' category:
  . Most correlated unigrams:
. stop
. lane
. car
. driver
. take
  . Most correlated bigrams:
. software discrepancy
. driver take

# 'DiDi Research America, LLC' category:
  . Most correlated unigrams:
. right
. perception
. software
. vehicle
. due
  . Most correlated bigrams:
. right turn
. perception discrepancy

# 'Gatik AI Inc.' category:
  . Most correlated unigrams:
. software
. plan
. perception
. behaviour
. motion
  . Most correlated bigrams:
. motion behaviour
. perception discrepancy

# 'Intel Corporation' category:
  . Most correlated unigrams:
. take
. driver
. map
. discrepancy
. software
 

As we can see, the unigrams correspond well to their category. However, bigrams do not. If we get the bigrams in our features:

In [42]:
bigrams

['perception discrepancy',
 'right turn',
 'deficiency track',
 'track module',
 'algorithmic deficiency',
 'motion behaviour',
 'behaviour plan',
 'plan discrepancy',
 'take car',
 'get close',
 'lane change',
 'manual lane',
 'make manual',
 'around car',
 'need manually',
 'manually drive',
 'stop lane',
 'take make',
 'sensor set',
 'deficiency sensor',
 'car stop',
 'drive around',
 'drive traffic',
 'traffic participant',
 'result gps',
 'gps system',
 'system give',
 'map discrepancy',
 'traffic condition',
 'give traffic',
 'unexpected result',
 'driver take',
 'reckless drive',
 'software discrepancy']

We can see there are only six. This means the unigrams have more correlation with the category than the bigrams, and since we're restricting the number of features to the most representative 300, only a few bigrams are being considered.

Let's save the files we'll need in the next steps:

In [44]:
# X_train
with open('Pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('Pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('Pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('Pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('Pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('Pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('Pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('Pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('Pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('Pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)